In [32]:
# install dependencies
!pip3 install snscrape
!pip install vaderSentiment
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
import pandas as pd
import io
from google.colab import files
import numpy as np
import pandas as pd

import snscrape.modules.twitter as sntwitter
import snscrape.modules.twitter

from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

import matplotlib.pyplot as mlpt
import csv
import datetime as dt
import pyspark as spark

import warnings
warnings.filterwarnings('ignore')
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col,udf,monotonically_increasing_id,unix_timestamp,round,avg
import re


In [34]:
uploaded = files.upload()


data = pd.read_csv(io.BytesIO(uploaded['tweets.csv']))
data_original = data
data

Saving tweets.csv to tweets (1).csv


,Unnamed: 0,Date,Content,User's location,# of likes
0,0,2023-02-16 23:48:21+00:00,#Turkey's #Erdogan will be digging his grave i...,Predicting the unpredictable!,3
1,1,2023-02-16 23:45:56+00:00,#Turkey's #Erdogan will be digging his grave i...,Predicting the unpredictable!,9
2,2,2023-02-16 23:44:19+00:00,"Mustafa, who was rescued tonight at the 261st ...",NaN,132
3,3,2023-02-16 23:35:00+00:00,#ICYMI: Dartmouth-based @ninelocksbrew is aimi...,"Enfield, N.S.",5
4,4,2023-02-16 23:30:16+00:00,‘What we need to ask ourselves is how Muslims ...,London,7
...,...,...,...,...,...
11206,11206,2023-02-06 12:31:17+00:00,Unbelievable: Another 6.0 earthquake shook the...,"Kyiv, Ukraine",142
11207,11207,2023-02-06 10:12:20+00:00,Sad news from the #TurkeySyriaEarthquake\n\n65...,England,8
11208,11208,2023-02-06 08:11:06+00:00,Praying 🙏🕯🙏🕯\nfor everyone affected by Deadly ...,"Phoenix, AZ",8
11209,11209,2023-02-06 05:02:06+00:00,#TurkeySyriaEarthquake is heartbreaking traged...,"Washington, DC",3


In [74]:
data['Date'].astype('str')
data['Date_new'] = [x[0:10] for x in data['Date']]

N = 1000

date_groups = data.groupby('Date_new').head(N).reset_index(drop=True)

date_groups.drop(["User's location", "# of likes", 'Date', 'Unnamed: 0'], axis = 1, inplace = True)

date_groups.head()

,Content,Date_new
0,#Turkey's #Erdogan will be digging his grave i...,2023-02-16
1,#Turkey's #Erdogan will be digging his grave i...,2023-02-16
2,"Mustafa, who was rescued tonight at the 261st ...",2023-02-16
3,#ICYMI: Dartmouth-based @ninelocksbrew is aimi...,2023-02-16
4,‘What we need to ask ourselves is how Muslims ...,2023-02-16


In [75]:
# sc = spark.SparkContext()
# sql = spark.SQLContext(sc)

In [76]:
def clean_tweet_content(tweets_df):
  data=tweets_df
  cdata=pd.DataFrame(columns=['content','date'])
  index=0
  for index,row in data.iterrows():
      stre=row["Content"]
      #the following cleaning steps are not the standard. You may generate your own ways of cleaning.
      # my_new_string = re.sub('#\w+', '', stre) #remove the hashtags
      # my_new_string = re.sub('http\S+', '', my_new_string) #remohbve the links

      my_new_string = re.sub('http\S+', '', stre) #remohbve the links

      my_new_string = re.sub('\n', '', my_new_string) #remove \n
      cdata.sort_index()
      cdata.at[index,'date']=row["Date_new"]
      cdata.at[index,'content']=my_new_string
      index=index+1
  tweets_df=cdata
  # pre_tweets_df['date'] = pre_tweets_df['date']

  # pre_tweets_df['date'] = pre_tweets_df['date'].astype('datetime64[ns]')
  # # pre_tweets_df["date"] = pre_tweets_df["date"].dt.date

  return tweets_df

In [77]:

def generateSentiment(tweets_df):

  FullDataTw=sql.createDataFrame(tweets_df)
  FullDataTw = FullDataTw.dropna()
  FullDataTw.select(monotonically_increasing_id().alias("rowId"),"*")

  CleanDF = FullDataTw
  from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
  analyser = SentimentIntensityAnalyzer()
  def senti_score_udf(sentence):
      snt = analyser.polarity_scores(sentence)
      return ([snt['neg'], snt['neu'], snt['pos'], snt['compound']])

  func_udf2 = udf(senti_score_udf, ArrayType(FloatType()))
  CleanDF = CleanDF.withColumn('p_neg', func_udf2(CleanDF['content'])[0])  #withColumn function would generate the result with a new column name
  CleanDF = CleanDF.withColumn('p_neu', func_udf2(CleanDF['content'])[1])
  CleanDF = CleanDF.withColumn('p_pos', func_udf2(CleanDF['content'])[2])
  CleanDF = CleanDF.withColumn('p_comp', func_udf2(CleanDF['content'])[3])
  CleanDF.show(10)
  # wc_tweets_df = CleanDF.toPandas().to_csv('sen.csv',index=False)
  tweets_df = CleanDF.toPandas()
  return tweets_df

In [78]:
date_groups = clean_tweet_content(date_groups)

In [79]:
date_groups_sentiment = generateSentiment(date_groups)

+--------------------+----------+-----+-----+-----+-------+
|             content|      date|p_neg|p_neu|p_pos| p_comp|
+--------------------+----------+-----+-----+-----+-------+
|#Turkey's #Erdoga...|2023-02-16|0.157|0.843|  0.0| -0.802|
|#Turkey's #Erdoga...|2023-02-16|0.157|0.843|  0.0| -0.802|
|Mustafa, who was ...|2023-02-16|  0.0|0.748|0.252| 0.8442|
|#ICYMI: Dartmouth...|2023-02-16|  0.0|0.926|0.074| 0.4019|
|‘What we need to ...|2023-02-16| 0.11|0.823|0.067|-0.4019|
|Join us and @worl...|2023-02-16|0.084|0.579|0.338| 0.7845|
|FAQs: Your donati...|2023-02-16|  0.0|  1.0|  0.0|    0.0|
|A father trying h...|2023-02-16|0.115|0.681|0.204|  0.658|
|@mayawiley A cat ...|2023-02-16|  0.0|0.667|0.333| 0.7906|
|So many people (i...|2023-02-16|0.286|0.667|0.048|  -0.91|
+--------------------+----------+-----+-----+-----+-------+
only showing top 10 rows



In [80]:
date_groups_sentiment.shape

(6925, 6)

In [81]:
from google.colab import files
date_groups_sentiment.to_csv('sentiment_tweets_df.csv', encoding = 'utf-8-sig') 
files.download('sentiment_tweets_df.csv')

# post_tweets_df.to_csv('post_tweets_df.csv', encoding = 'utf-8-sig') 
# files.download('post_tweets_df.csv')

NameError: ignored